In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from __future__ import print_function
%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec




manualSeed = 999

print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:
imagefolder = '/kaggle/input/celeba-dataset/img_align_celeba'
print(os.listdir(imagefolder))

out_folder = "/kaggle/working/models/celeba/"

batch_size = 128
image_size = 64
nc = 3
noise_dim = 100
num_epochs = 0
lr = 0.0002
beta1 = 0.5
iter_check = 2000
print_check = 100
demo_batch_size = 5
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print(device)

NUM_TRAIN = 200000

In [ ]:
os.mkdir("/kaggle/working/models")
os.mkdir("/kaggle/working/models/celeba")
os.mkdir("/kaggle/working/models/dataloader")


In [ ]:
dataset = dset.ImageFolder(root=imagefolder,transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

print(len(dataset))

In [ ]:
mask = [i for i in range(NUM_TRAIN)]
print(mask[0])
training_set = torch.utils.data.Subset(dataset,mask)

In [ ]:
val_mask  = [i for i in range(NUM_TRAIN,len(dataset))]
print(val_mask[-1])
val_set = torch.utils.data.Subset(dataset,val_mask)

In [ ]:
import os
print(os.listdir("/kaggle/working/"))

In [ ]:
val_loader = torch.utils.data.DataLoader(val_set, batch_size=32,
                                         shuffle=True,drop_last=True ,num_workers=4,pin_memory=True)
print(len(val_loader))

torch.save(val_loader,"/kaggle/working/models/dataloader/test.pt")

In [ ]:
os.listdir("/kaggle/working/models/dataloader")

In [ ]:
print(len(training_set))

In [ ]:

# Create the dataloader
dataloader = torch.utils.data.DataLoader(training_set, batch_size=batch_size,
                                         shuffle=True,drop_last=True ,num_workers=4,pin_memory=True)
print(len(dataloader))

In [ ]:
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [ ]:
from models import Generator,Discriminator
from helpers import initialize_weights,get_optimizer
Gen = Generator().to(device)
Dis = Discriminator().to(device) 

Gen.apply(initialize_weights)
Dis.apply(initialize_weights)

criterion = nn.BCEWithLogitsLoss()

Gen_opt = get_optimizer(Gen)
Dis_opt = get_optimizer(Dis)

random_images = []
fixed_noise = torch.randn(64, noise_dim, 1, 1, device=device)

iters = 0
G_losses = []
D_losses = []

In [ ]:
num_epochs=25
print(iter_check)
iter_check=1000
print(num_epochs)
print(batch_size)

In [ ]:
epochs_done = 25

In [ ]:
new_epochs = 2
num_epochs = epochs_done + new_epochs
for epoch in range(epochs_done,num_epochs):
    for i,data in enumerate(dataloader,0):
        Dis_opt.zero_grad()
        data = data[0]
        real_images = data.to(device)
        b_size=real_images.size(0)

        d_label_real=0.9*torch.ones(b_size,device=device) + 0.1*torch.rand(b_size,device=device)
        d_real = Dis(real_images).view(-1)
        d_loss_real = criterion(d_real,d_label_real)

        d_noise=torch.randn(b_size,noise_dim, 1, 1,device=device)
        d_fake_images = Gen(d_noise).detach()
        d_fake = Dis(d_fake_images).view(-1)
        d_fake_label = torch.zeros(b_size,device=device)
        d_loss_fake = criterion(d_fake,d_fake_label)

        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()

        D_x = d_real.mean().item()
        D_G_z1 = d_fake.mean().item()

        Dis_opt.step()

        Gen_opt.zero_grad()
        g_noise=torch.randn(b_size,noise_dim, 1, 1,device=device)
        g_fake_images=Gen(g_noise)

        g_label = torch.ones(b_size,device=device)

        g_fake_images = Dis(g_fake_images).view(-1)
        g_loss = criterion(g_fake_images,g_label)
        g_loss.backward()
        D_G_z2 = g_fake_images.mean().item()

        Gen_opt.step()

        if i % print_check == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     d_loss.item(), g_loss.item(), D_x, D_G_z1, D_G_z2))
        
        # Save Losses for plotting later
        G_losses.append(g_loss.item())
        D_losses.append(d_loss.item())
        
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % iter_check == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = Gen(fixed_noise).detach().cpu()
            random_images.append(vutils.make_grid(fake, padding=2, normalize=True))
            
        iters += 1
    curr_outpath=os.path.join(out_folder,"celeba_epoch{}.pt".format(epoch+1))
    torch.save({"epoch":epoch+1,"state_dict_G":Gen.state_dict(),
                "state_dict_D":Dis.state_dict(),
                "opt_G":Gen_opt.state_dict(),
                "opt_D":Dis_opt.state_dict()
    },curr_outpath)
    epochs_done += 1
    print(epoch+1)
    plt.subplot(1,1,1)
    plt.axis("off")
    plt.title("Images")
    plt.imshow(np.transpose(random_images[-1],(1,2,0)))
    plt.show()


In [ ]:
fixed_noise = torch.randn(16, noise_dim, 1, 1, device=device)


In [ ]:
new_epochs = 1
num_epochs = epochs_done + new_epochs
for epoch in range(epochs_done,num_epochs):
    for i,data in enumerate(dataloader,0):
        Dis_opt.zero_grad()
        data = data[0]
        real_images = data.to(device)
        b_size=real_images.size(0)

        d_label_real=0.9*torch.ones(b_size,device=device) + 0.1*torch.rand(b_size,device=device)
        d_real = Dis(real_images).view(-1)
        d_loss_real = criterion(d_real,d_label_real)

        d_noise=torch.randn(b_size,noise_dim, 1, 1,device=device)
        d_fake_images = Gen(d_noise).detach()
        d_fake = Dis(d_fake_images).view(-1)
        d_fake_label = torch.zeros(b_size,device=device)
        d_loss_fake = criterion(d_fake,d_fake_label)

        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()

        D_x = d_real.mean().item()
        D_G_z1 = d_fake.mean().item()

        Dis_opt.step()

        Gen_opt.zero_grad()
        g_noise=torch.randn(b_size,noise_dim, 1, 1,device=device)
        g_fake_images=Gen(g_noise)

        g_label = torch.ones(b_size,device=device)

        g_fake_images = Dis(g_fake_images).view(-1)
        g_loss = criterion(g_fake_images,g_label)
        g_loss.backward()
        D_G_z2 = g_fake_images.mean().item()

        Gen_opt.step()

        if i % print_check == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     d_loss.item(), g_loss.item(), D_x, D_G_z1, D_G_z2))
        
        # Save Losses for plotting later
        G_losses.append(g_loss.item())
        D_losses.append(d_loss.item())
        
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % iter_check == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = Gen(fixed_noise).detach().cpu()
            random_images.append(vutils.make_grid(fake, padding=2, normalize=True))
            
        iters += 1
    curr_outpath=os.path.join(out_folder,"celeba_epoch{}.pt".format(epoch+1))
    torch.save({"epoch":epoch+1,"state_dict_G":Gen.state_dict(),
                "state_dict_D":Dis.state_dict(),
                "opt_G":Gen_opt.state_dict(),
                "opt_D":Dis_opt.state_dict()
    },curr_outpath)
    epochs_done += 1
    print(epoch+1)
    plt.axis("off")
    plt.title("Images")
    plt.imshow(np.transpose(random_images[-1],(1,2,0)))
    plt.show()


In [ ]:
plt.axis("off")
plt.title("Images")
plt.imshow(np.transpose(random_images[-1],(1,2,0)))
plt.show()

In [ ]:
with torch.no_grad():
     fake = Gen(fixed_noise).detach().cpu()
random_images.append(vutils.make_grid(fake, padding=2, normalize=True))

In [ ]:
dataloader = torch.utils.data.DataLoader(training_set, batch_size=32,
                                         shuffle=True,drop_last=True ,num_workers=4,pin_memory=True)
print(len(dataloader))

In [ ]:
new_epochs = 1
num_epochs = epochs_done + new_epochs
for epoch in range(epochs_done,num_epochs):
    for i,data in enumerate(dataloader,0):
        Dis_opt.zero_grad()
        data = data[0]
        real_images = data.to(device)
        
        b_size=real_images.size(0)

        d_label_real=0.9*torch.ones(b_size,device=device) + 0.1*torch.rand(b_size,device=device)
        d_real = Dis(real_images).view(-1)
        d_loss_real = criterion(d_real,d_label_real)

        d_noise=torch.randn(b_size,noise_dim, 1, 1,device=device)
        d_fake_images = Gen(d_noise).detach()
        d_fake = Dis(d_fake_images).view(-1)
        d_fake_label = torch.zeros(b_size,device=device)
        d_loss_fake = criterion(d_fake,d_fake_label)

        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()

        D_x = d_real.mean().item()
        D_G_z1 = d_fake.mean().item()

        Dis_opt.step()

        Gen_opt.zero_grad()
        g_noise=torch.randn(b_size,noise_dim, 1, 1,device=device)
        g_fake_images=Gen(g_noise)

        g_label = torch.ones(b_size,device=device)

        g_fake_images = Dis(g_fake_images).view(-1)
        g_loss = criterion(g_fake_images,g_label)
        g_loss.backward()
        D_G_z2 = g_fake_images.mean().item()

        Gen_opt.step()

        if i % print_check == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     d_loss.item(), g_loss.item(), D_x, D_G_z1, D_G_z2))
            print(real_images.size())
        
        # Save Losses for plotting later
        G_losses.append(g_loss.item())
        D_losses.append(d_loss.item())
        
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % iter_check == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = Gen(fixed_noise).detach().cpu()
            random_images.append(vutils.make_grid(fake, padding=2, normalize=True))
            
        iters += 1
    curr_outpath=os.path.join(out_folder,"celeba_epoch{}.pt".format(epoch+1))
    torch.save({"epoch":epoch+1,"state_dict_G":Gen.state_dict(),
                "state_dict_D":Dis.state_dict(),
                "opt_G":Gen_opt.state_dict(),
                "opt_D":Dis_opt.state_dict()
    },curr_outpath)
    epochs_done += 1
    print(epoch+1)
    plt.axis("off")
    plt.title("Images")
    plt.imshow(np.transpose(random_images[-1],(1,2,0)))
    plt.show()


In [ ]:
from matplotlib.lines import Line2D
def plot_grad_flow(named_parameters):
    '''Plots the gradients flowing through different layers in the net during training.
    Can be used for checking for possible gradient vanishing / exploding problems.
    
    Usage: Plug this function in Trainer class after loss.backwards() as 
    "plot_grad_flow(self.model.named_parameters())" to visualize the gradient flow'''
    ave_grads = []
    max_grads= []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean())
            max_grads.append(p.grad.abs().max())
    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
    plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom = -0.001, top=0.05) # zoom in on the lower gradient regions
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    plt.legend([Line2D([0], [0], color="c", lw=4),
                Line2D([0], [0], color="b", lw=4),
                Line2D([0], [0], color="k", lw=4)], ['max-gradient', 'mean-gradient', 'zero-gradient'])

In [ ]:
plot_grad_flow(Dis.named_parameters())

In [ ]:
class ImageMask(torch.utils.data.Dataset):
  def __init__(self, dataset, image_size=(3,64,64),param=0.4):
    self.dataset = dataset
    self.image_size = image_size
    self.image_shape = (image_size[1],image_size[2])
    self.map = ['random','center','left','right','grid','up','down']
    self.param = param
  
  def __getitem__(self, index):
    target_image = self.dataset[index][0]
    assert(torch.is_tensor(target_image))
    
    maskType = self.map[index%2]
    mask = np.ones(self.image_shape)

    param = 0.4
    if maskType == 'random':
        mask[np.random.random(self.image_shape) < param] = 0.0
    elif maskType == 'center':
        centerScale = 0.35
        sz = tuple([(int)(z * centerScale) for z in self.image_shape])
        mask[ sz[1]:-sz[1], sz[0]:-sz[0]] = 0.0
    elif maskType == 'left':
        c = self.image_shape[1] // 2
        mask[:c,:] = 0.0
    elif maskType == 'right':
        c = self.image_shape[1] // 2
        mask[c:,:] = 0.0
    elif maskType == 'grid':
        mask[::4,::4] = 0.0
    elif maskType == 'up':
        c = self.image_shape[0] // 2
        mask[:,:c] = 0.0
    elif maskType == 'down':
        c = self.image_shape[0] // 2
        mask[:,c:] = 0.0
    else:
        assert(False)

    return (target_image, torch.FloatTensor(mask))
  
  def __len__(self):
    return len(self.dataset)

In [ ]:
masked_dataset = ImageMask(val_set)
masked_loader = torch.utils.data.DataLoader(masked_dataset, batch_size=64,
                                         shuffle=False,drop_last=True ,num_workers=0,pin_memory=True)

In [ ]:
from models import Generator,Discriminator

Gen_eval = Generator().to(device)
Dis_eval = Discriminator().to(device) 

Gen_eval.load_state_dict(Gen.state_dict())
Dis_eval.load_state_dict(Dis.state_dict())

Gen_eval.eval()
Dis_eval.eval()



In [ ]:
from models import Generator,Discriminator

pretrained = torch.load("../input/pretrained/celeba_epoch31.pt")
Gen_eval = Generator().to(device)
Dis_eval = Discriminator().to(device) 

Gen_eval.load_state_dict(pretrained["state_dict_G"])
Dis_eval.load_state_dict(pretrained["state_dict_D"])

Gen_eval.eval()
Dis_eval.eval()

In [ ]:
for p in Gen_eval.parameters():
    p.requires_grad=False
    
for p in Dis_eval.parameters():
    p.requires_grad=False
    

In [ ]:
WINDOW_SIZE = 7
prior_loss_parameter = 0.003
special_conv = torch.ones(1,1,WINDOW_SIZE,WINDOW_SIZE,device=device)
inpaint_iters = 1

In [ ]:
def image_gradient(image,device):
    grad_mat = torch.Tensor([[[[1, 0, -1], [2, 0, -2], [1, 0, -1]]]]).to(device)
    image_size = image.size()
    image = image.view(-1, 1, image_size[2], image_size[3])
    grad_x = torch.nn.functional.conv2d(image, grad_mat, padding=1)
    grad_y = torch.nn.functional.conv2d(image, torch.transpose(grad_mat, 2, 3), padding=1)
    return grad_x.view(image_size), grad_y.view(image_size)

In [ ]:
def posisson_blending(masks, masks_inv, generated_images, corrupted_images,device ,blending_steps=3000):
  #reference: https://zhuanlan.zhihu.com/p/51279118
    initial_guess = masks*corrupted_images + (masks_inv)*generated_images
    optimum = initial_guess.requires_grad_()
    optimizer = torch.optim.Adam([optimum])
    generated_grad_x, generated_grad_y = image_gradient(generated_images,device)

    for epoch in range(blending_steps):
        optimizer.zero_grad()
        image_optimum_grad_x, image_optimum_grad_y = image_gradient(optimum,device)
        loss = torch.sum(((generated_grad_x-image_optimum_grad_x)**2 + (generated_grad_y-image_optimum_grad_y)**2)*(masks_inv))
        loss.backward()
        optimum.grad = optimum.grad*(masks_inv) #maintaining fixed pixel values
        optimizer.step()

    return optimum.detach()

In [ ]:
import matplotlib.pyplot as plt
inpaint_iters=1500
for i,data in enumerate(masked_loader,0):
    
    b_size = data[0].size(0)
    image_batch = data[0].to(device)
    masks = data[1].to(device).unsqueeze(1)
    masks_inv = torch.ones_like(masks) - masks
    
    weighted_masks = (torch.nn.functional.conv2d(masks_inv,special_conv,padding=WINDOW_SIZE//2)*masks + masks)/(WINDOW_SIZE*WINDOW_SIZE)

    z_closest = torch.randn(b_size,noise_dim,1,1,device=device,requires_grad=True)
    z_optimizer = torch.optim.Adam([z_closest])
    
    prior_criterior = nn.BCEWithLogitsLoss()
    
    for i in range(inpaint_iters):
        z_optimizer.zero_grad()
        fake_images = Gen_eval(z_closest)
        d_output = Dis_eval(fake_images).view(-1)

        prior_loss=prior_loss_parameter*prior_criterior(d_output,torch.ones(b_size,device=device))

        context_loss = torch.norm(weighted_masks*(fake_images - image_batch),p=1)
        loss = context_loss + prior_loss
        loss.backward()
        z_optimizer.step()
    inpainted_images_gan = Gen_eval(z_closest.detach()).detach()    
    inpainted_images = posisson_blending(masks,masks_inv,inpainted_images_gan,image_batch,device,blending_steps=1000)
    
    for img,gen_img,genp_img,mask in zip(image_batch,inpainted_images_gan,inpainted_images,masks):
        og = transforms.ToPILImage(mode="RGB")(img.cpu()/2 + 0.5)
        inp = transforms.ToPILImage(mode="RGB")(gen_img.cpu()/2 + 0.5)
        inpp = transforms.ToPILImage(mode="RGB")((genp_img/2 + 0.5).cpu())
        ogm = transforms.ToPILImage(mode="RGB")(((img/2 + 0.5)*mask).cpu())
        inpm = transforms.ToPILImage(mode="RGB")(((gen_img/2 + 0.5)*mask).cpu())
        inppm = transforms.ToPILImage(mode="RGB")(((genp_img/2 + 0.5)*mask).cpu())
        pinp = transforms.ToPILImage(mode="RGB")(((img/2 + 0.5)*mask + (gen_img/2 + 0.5)*(1-mask)).cpu())
        pinpm = transforms.ToPILImage(mode="RGB")(((img/2 + 0.5)*mask).cpu())
        f = plt.figure()
        f.add_subplot(2,4, 1)
        plt.imshow(og)
        f.add_subplot(2,4, 2)
        plt.imshow(ogm)
        f.add_subplot(2,4, 3)
        plt.imshow(inp)
        f.add_subplot(2,4, 4)
        plt.imshow(inpm)
        f.add_subplot(2,4, 5)
        plt.imshow(inpp)
        f.add_subplot(2,4, 6)
        plt.imshow(inppm)
        f.add_subplot(2,4, 7)
        plt.imshow(pinp)
        f.add_subplot(2,4, 8)
        plt.imshow(pinpm)
        plt.show(block=True)
        
    break
        
    
        